# Game Recommendation Data Retrieval

In [1]:
import time
import csv
import requests
from queue import Queue
from threading import Thread

key = "KEY GOES HERE" # Free to get

### Single Thread
#### Only do if you can't do multithread, will take longer

In [6]:
start = time.time()

# Set up the API call parameters
url = "https://api.rawg.io/api/games"
params = {
    "page": 1,
    "page_size": 5,
    "key": key
}

fields = ["id", "name", "released", 
          "rating", 
          "platforms", "series", "genre",
          "tags","metacritic","playtime",]

# Open the CSV file for writing
with open("games.csv", mode="w", newline="", encoding="utf-8") as file:
    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(fields)

    # Loop through the pages of the API results
    for page in range(1, 2):
        # Set the page parameter in the API call
        params["page"] = page

        # Make the API call and get the JSON response
        response = requests.get(url, params=params)
        data = response.json()["results"]
        print(page)
        
            # Use the csv module to write the games data to the file
        csv.writer(file).writerows([[game["id"], game["name"], game["released"], 
                                     game["rating"], 
                                     "-".join([platform["platform"]["name"] for platform in game["platforms"]]) if game["platforms"] else "", 
                                     "-".join([str(game_data["id"]) for game_data in requests.get(f"https://api.rawg.io/api/games/{game['id']}/game-series", params={"key": params["key"]}).json()["results"]]) if requests.get(f"https://api.rawg.io/api/games/{game['id']}/game-series", params={"key": params["key"]}).json()["results"] else "",
                                     "-".join([genre["name"] for genre in game["genres"]]) if game["genres"] else "",
                                     "-".join([tag["name"] for tag in game["tags"]]) if game["tags"] else "",
                                     game["metacritic"],
                                     game["playtime"]] for game in data])
            

end = time.time()
print(f"Elapsed time: {end - start} seconds")

1
Elapsed time: 6.639714956283569 seconds


### Multi Thread
#### Yeah, pretty cool :d

In [8]:
start = time.time()

# Set up the API call parameters
url = "https://api.rawg.io/api/games"
params = {
    "page": 1,
    "page_size": 40,
    "key": "f2d896f890e746e6bdf1bdf8d1391e6a"
}

fields = ["id", "name", "released", 
          "rating", 
          "platforms", "series", "genre",
          "tags","metacritic","playtime",]

# Define a worker function that takes items from the queue and writes them to the CSV file
def worker(q, file):
    while True:
        # Get an item from the queue
        item = q.get()
        # If the item is None, break the loop
        if item is None:
            break
        # Otherwise, write the item to the file
        writer = csv.writer(file)
        writer.writerow(item)
        # Indicate that the task is done
        q.task_done()

# Open the CSV file for writing
with open("games2.csv", mode="w", newline="", encoding="utf-8") as file:
    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(fields)

    # Create a queue object
    q = Queue()

    # Create a list of threads
    num_threads = 8 # You can adjust this number according to your needs
    threads = []

    # Start num_threads threads and add them to the list
    for i in range(num_threads):
        t = Thread(target=worker, args=(q, file))
        t.start()
        threads.append(t)

    # Loop through the pages of the API results
    for page in range(53, 250):
        # Set the page parameter in the API call
        params["page"] = page

        # Make the API call and get the JSON response
        response = requests.get(url, params=params)
        data = response.json()["results"]
        
        # Put each game data into the queue as a list
        for game in data:
            q.put([game["id"], game["name"], game["released"], 
                   game["rating"], 
                   "-".join([platform["platform"]["name"] for platform in game["platforms"]]) if game["platforms"] else "", 
                   "-".join([str(game_data["id"]) for game_data in requests.get(f"https://api.rawg.io/api/games/{game['id']}/game-series", params={"key": params["key"]}).json()["results"]]) if requests.get(f"https://api.rawg.io/api/games/{game['id']}/game-series", params={"key": params["key"]}).json()["results"] else "",
                   "-".join([genre["name"] for genre in game["genres"]]) if game["genres"] else "",
                   "-".join([tag["name"] for tag in game["tags"]]) if game["tags"] else "",
                   game["metacritic"],
                   game["playtime"]])

    # Put None into the queue for each thread to indicate that there is no more data
    for i in range(num_threads):
        q.put(None)

    # Wait for all threads to finish
    for t in threads:
        t.join()
        
end = time.time()
print(f"Elapsed time: {end - start} seconds")

Elapsed time: 2762.350819349289 seconds
